In [1]:
!pip install python-dwca-reader requests tqdm python-dotenv beautifulsoup4 aiohttp
from dotenv import load_dotenv

load_dotenv()

True

In [24]:
import os
from datetime import datetime

import requests
from dwca.read import DwCAReader
from tqdm import tqdm

worms_url = os.environ.get("WORMS_DWCA_ARCHIVE")
worms_url = worms_url.replace("{year}", str(datetime.now().year)).replace(
    "{full_date}", datetime.now().strftime("%Y-%m-01")
)
worms_login = os.environ.get("WORMS_LOGIN")
worms_password = os.environ.get("WORMS_PASSWORD")


def download(url: str, filename: str, auth: tuple = None, chunk_size: int = 1024) -> None:
    with open(filename, "wb") as f, requests.get(url, stream=True, auth=auth) as r:
        r.raise_for_status()
        total = int(r.headers.get("content-length", 0))

        tqdm_params = {
            "desc": url,
            "total": total,
            "miniters": 1,
            "unit": "B",
            "unit_scale": True,
            "unit_divisor": 1024,
        }
        total_downloaded = 0
        with tqdm(**tqdm_params) as pb:
            for chunk in r.iter_content(chunk_size=chunk_size):
                pb.update(len(chunk))
                f.write(chunk)
                total_downloaded += len(chunk)

        # Compare the resulting length
        if total and total_downloaded != total:
            raise ValueError(f"Downloaded file size mismatch: expected {total} bytes, got {total_downloaded} bytes")
        elif total == 0:
            print("Warning: Content-Length header not provided, cannot verify the file size.")


download(worms_url, "worms.zip", auth=(worms_login, worms_password), chunk_size=8192)


with DwCAReader("worms.zip") as dwca:
    print(f"Core data file is: {dwca.descriptor.core.file_location}")

    worms = dwca.pd_read(dwca.descriptor.core.file_location, parse_dates=True)
    worms.head()

worms

https://www.marinespecies.org/export/worms/2024/WoRMS_download_2024-09-01.zip: 100%|██████████| 57.0M/57.0M [00:06<00:00, 8.63MB/s]


Core data file is: taxon.txt


,id,scientificNameID,acceptedNameUsageID,parentNameUsageID,namePublishedInID,scientificName,acceptedNameUsage,parentNameUsage,namePublishedIn,namePublishedInYear,...,taxonomicStatus,nomenclaturalStatus,modified,bibliographicCitation,references,license,rightsHolder,datasetName,institutionCode,datasetID
0,urn:lsid:marinespecies.org:taxname:1,urn:lsid:marinespecies.org:taxname:1,urn:lsid:marinespecies.org:taxname:1,NaN,NaN,Biota,Biota,Biota,NaN,NaN,...,accepted,NaN,2004-12-21,WoRMS (2024). Biota. Accessed at: https://www.marinespecies.org/aphia.php?p=taxdetails&id=1,https://www.marinespecies.org/aphia.php?p=taxdetails&id=1,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170
1,urn:lsid:marinespecies.org:taxname:2,urn:lsid:marinespecies.org:taxname:2,urn:lsid:marinespecies.org:taxname:2,urn:lsid:marinespecies.org:taxname:1,NaN,Animalia,Animalia,Biota,NaN,NaN,...,accepted,NaN,2004-12-21,WoRMS (2024). Animalia. Accessed at: https://www.marinespecies.org/aphia.php?p=taxdetails&id=2,https://www.marinespecies.org/aphia.php?p=taxdetails&id=2,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170
2,urn:lsid:marinespecies.org:taxname:3,urn:lsid:marinespecies.org:taxname:3,urn:lsid:marinespecies.org:taxname:3,urn:lsid:marinespecies.org:taxname:1,NaN,Plantae,Plantae,Biota,NaN,NaN,...,accepted,NaN,2011-07-29,WoRMS (2024). Plantae. Accessed at: https://www.marinespecies.org/aphia.php?p=taxdetails&id=3,https://www.marinespecies.org/aphia.php?p=taxdetails&id=3,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170
3,urn:lsid:marinespecies.org:taxname:4,urn:lsid:marinespecies.org:taxname:4,urn:lsid:marinespecies.org:taxname:4,urn:lsid:marinespecies.org:taxname:1,NaN,Fungi,Fungi,Biota,NaN,NaN,...,accepted,NaN,2004-12-21,WoRMS (2024). Fungi. Accessed at: https://www.marinespecies.org/aphia.php?p=taxdetails&id=4,https://www.marinespecies.org/aphia.php?p=taxdetails&id=4,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170
4,urn:lsid:marinespecies.org:taxname:5,urn:lsid:marinespecies.org:taxname:5,urn:lsid:marinespecies.org:taxname:5,urn:lsid:marinespecies.org:taxname:1,NaN,Protozoa,Protozoa,Biota,NaN,NaN,...,accepted,NaN,2011-07-29,WoRMS (2024). Protozoa. Accessed at: https://www.marinespecies.org/aphia.php?p=taxdetails&id=5,https://www.marinespecies.org/aphia.php?p=taxdetails&id=5,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579151,urn:lsid:marinespecies.org:taxname:1768285,urn:lsid:marinespecies.org:taxname:1768285,urn:lsid:marinespecies.org:taxname:446505,urn:lsid:marinespecies.org:taxname:1768284,NaN,Turritella (Ctenocolpus) australis,Colpospira australis,Turritella (Ctenocolpus),NaN,NaN,...,superseded combination,NaN,2024-08-31,"MolluscaBase eds. (2024). MolluscaBase. Turritella (Ctenocolpus) australis Lamarck, 1822. Accessed through: World Register of Marine Species at: https://www.marinespecies.org/aphia.php?p=taxdetails&id=1768285",https://www.marinespecies.org/aphia.php?p=taxdetails&id=1768285,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170
579152,urn:lsid:marinespecies.org:taxname:1768292,urn:lsid:marinespecies.org:taxname:1768292,NaN,urn:lsid:marinespecies.org:taxname:160366,NaN,Melampus brotianus,NaN,Melampus,"Pfeiffer, L. (1860). Zwei neue Auriculaceen. <em>Malakozoologische Blätter.</em> 6: 201-202.",1860,...,unassessed,NaN,2024-08-31,"MolluscaBase eds. (2024). MolluscaBase. Melampus brotianus L. Pfeiffer, 1860. Accessed t

In [4]:
download(worms_url, "worms.zip", auth=(worms_login, worms_password))

https://www.marinespecies.org/export/worms/2024/WoRMS_download_2024-09-01.zip:   0%|          | 196k/57.0M [00:07<35:26, 28.0kB/s]   


KeyboardInterrupt: 

In [5]:
obis_url = os.getenv("OBIS_PARQUET_URL")

download(obis_url, "obis.parquet", chunk_size=8192)

https://obis-datasets.ams3.digitaloceanspaces.com/exports/obis_20240723.parquet:   0%|          | 84.9M/18.3G [00:17<1:02:49, 5.19MB/s]


KeyboardInterrupt: 

In [5]:
invasive = os.getenv("INVASIVE_DWCA_ARCHIVE")

download(invasive, "invasive.zip")


with DwCAReader("invasive.zip") as dwca:
    print(f"Core data file is: {dwca.descriptor.core.file_location}")

    worms = dwca.pd_read(dwca.descriptor.core.file_location, parse_dates=True)

worms

https://ipt.gbif.org/archive.do?r=gisd&v=1.4: 307kB [00:00, 424kB/s]  


Core data file is: taxon.txt


,id,taxonID,scientificName,kingdom,phylum,class,order,family,genus,scientificNameAuthorship,taxonomicStatus,datasetName,references
0,2,2,Miconia calvescens D.C.,Plantae,Magnoliophyta,Magnoliopsida,Myrtales,Melastomataceae,Miconia,D.C.,accepted,Global Invasive Species Database,http://www.issg.org/database/species/ecology.a...
1,15,15,Abelmoschus moschatus Medik.,Plantae,Magnoliophyta,Magnoliopsida,Fabales,Malvaceae,Abelmoschus,Medik.,accepted,Global Invasive Species Database,http://www.issg.org/database/species/ecology.a...
2,16,16,Imperata cylindrica (L.) Raeusch.,Plantae,Magnoliophyta,Liliopsida,Cyperales,Poaceae,Imperata,(L.) Raeusch.,accepted,Global Invasive Species Database,http://www.issg.org/database/species/ecology.a...
3,18,18,"Oryctolagus cuniculus (Linnaeus, 1758)",Animalia,Chordata,Mammalia,Lagomorpha,Leporidae,Oryctolagus,"(Linnaeus, 1758)",accepted,Global Invasive Species Database,http://www.issg.org/database/species/ecology.a...
4,19,19,"Rattus rattus (Linnaeus, 1758)",Animalia,Chordata,Mammalia,Rodentia,Muridae,Rattus,"(Linnaeus, 1758)",accepted,Global Invasive Species Database,http://www.issg.org/database/species/ecology.a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4949,1004008,1004008,Silurus gariepinus Burchell 1822,NaN,NaN,NaN,NaN,NaN,NaN,Burchell 1822,synonym,Global Invasive Species Database,http://www.issg.org/database/species/ecology.a...
4950,1004009,1004009,Sargassum hystrix var. fluitans B rgesen 1914,NaN,NaN,NaN,NaN,NaN,NaN,var. fluitans B rgesen 1914,synonym,Global Invasive Species Database,http://www.issg.org/database/species/ecology.a...
4951,1004010,1004010,Fucus baccifer var. oblongifolius Turner 1807,NaN,NaN,NaN,NaN,NaN,NaN,var. oblongifolius Turner 1807,synonym,Global Invasive Species Database,http://www.issg.org/database/species/ecology.a...
4952,1004011,1004011,"Mustela vison Schreber, 1777",NaN,NaN,NaN,NaN,NaN,NaN,"Schreber, 1777",synonym,Global Invasive Species Database,http://www.issg.org/database/species/ecology.a...


In [25]:
import asyncio

import aiohttp
import nest_asyncio
import pandas as pd
from bs4 import BeautifulSoup

# Allow nested event loops in Jupyter notebooks
nest_asyncio.apply()


# Asynchronous function to get common name and image URL from Wikipedia for a single species
async def get_species_info_from_wikipedia(scientific_name):
    try:
        async with aiohttp.ClientSession() as session:
            # Search Wikipedia for the species page
            search_url = (
                f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={scientific_name}&format=json"
            )
            async with session.get(search_url) as search_response:
                if search_response.status != 200:  # Corrected status check for aiohttp
                    return None, None
                search_data = await search_response.json()

            # Check if any page is found
            if not search_data["query"]["search"]:
                return None, None

            # Get the page title from the first search result
            page_title = search_data["query"]["search"][0]["title"]

            # Wikipedia API endpoint for fetching page info including image and extract
            page_url = f"https://en.wikipedia.org/w/api.php?action=query&titles={page_title}&prop=pageimages|extracts&format=json&pithumbsize=1024&exintro"
            async with session.get(page_url) as page_response:
                if page_response.status != 200:  # Corrected status check for aiohttp
                    return None, None
                page_data = await page_response.json()

            pages = page_data.get("query", {}).get("pages", {})

            if not pages:
                return None, None

            # Extract the page details
            page_info = next(iter(pages.values()))

            # Extract the common name from the first sentence of the extract
            extract_text = page_info.get("extract", "")
            image_url = page_info.get("thumbnail", {}).get("source", "Image not available")

            common_name = None
            if extract_text:
                # Use the first sentence of the extract as the common name
                first_sentence = extract_text.split(".")[0]

                # Remove HTML tags using BeautifulSoup
                common_name = BeautifulSoup(first_sentence, "html.parser").get_text().strip()

            return common_name, image_url
    except Exception as e:
        print(f"Error for {scientific_name}: {e}")
        return None, None


# Asynchronous function to process a DataFrame row and apply the Wikipedia info extraction
async def process_species_list(df, column_name):
    tasks = []
    for name in df[column_name]:
        # Create tasks for each species name
        tasks.append(get_species_info_from_wikipedia(name))

    # Gather all tasks (concurrent execution)
    results = await asyncio.gather(*tasks)

    # Assign the results to new columns in the DataFrame
    df[["Common Name", "Image URL"]] = pd.DataFrame(results, columns=["Common Name", "Image URL"])
    return df


# Function to run the async loop and process the DataFrame in Jupyter
async def asapply_wikipedia_info_to_df(df, column_name):
    # Process the DataFrame asynchronously
    return await process_species_list(df, column_name)

In [4]:
# Function to get common name and image URL from Wikipedia for a single species
def get_species_info_from_wikipedia(scientific_name):
    try:
        # Search Wikipedia for the species page
        search_url = (
            f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={scientific_name}&format=json"
        )
        search_response = requests.get(search_url)

        if search_response.status_code != 200:
            return None, None

        search_data = search_response.json()

        # Check if any page is found
        if not search_data["query"]["search"]:
            return None, None

        # Get the page title from the first search result
        page_title = search_data["query"]["search"][0]["title"]

        # Wikipedia API endpoint for fetching page info including image and extract
        page_url = f"https://en.wikipedia.org/w/api.php?action=query&titles={page_title}&prop=pageimages|extracts&format=json&pithumbsize=500&exintro"
        page_response = requests.get(page_url)

        if page_response.status_code != 200:
            return None, None
        page_data = page_response.json()
        pages = page_data.get("query", {}).get("pages", {})

        if not pages:
            return None, None

        # Extract the page details
        page_info = next(iter(pages.values()))

        # Extract the common name from the first sentence of the extract
        extract_text = page_info.get("extract", "")
        image_url = page_info.get("thumbnail", {}).get("source", "Image not available")

        common_name = None
        if extract_text:
            # Use the first sentence of the extract as the common name
            first_sentence = extract_text.split(".")[0]

            # Remove HTML tags using BeautifulSoup
            common_name = BeautifulSoup(first_sentence, "html.parser").get_text().strip()

        return common_name, image_url
    except:
        return None, None


# Function to apply the Wikipedia info extraction to a DataFrame
def apply_wikipedia_info_to_df(df, column_name):
    # Apply the species info function to each species in the specified column
    df[["Common Name", "Image URL"]] = df[column_name].apply(
        lambda name: pd.Series(get_species_info_from_wikipedia(name))
    )
    return df

In [5]:
import time

time_start = time.time()
limit_worms = worms.head(100)
# Apply the function to the DataFrame
updated_species_df = apply_wikipedia_info_to_df(limit_worms, "scientificName")

time_end = time.time()
time_elapsed = time_end - time_start
print(f"Time elapsed: {time_elapsed:.2f} seconds")
# Show the updated DataFrame
updated_species_df

Asking for Biota and getting 200 and 200
Asking for Animalia and getting 200 and 200
Asking for Plantae and getting 200 and 200
Asking for Fungi and getting 200 and 200
Asking for Protozoa and getting 200 and 200
Asking for Bacteria and getting 200 and 200
Asking for Chromista and getting 200 and 200
Asking for Archaea and getting 200 and 200
Asking for Sarcomastigophora and getting 200 and 200
Asking for Viruses and getting 200 and 200
Asking for Ciliophora and getting 200 and 200
Asking for Biota incertae sedis and getting 200 and 200
Asking for Mastigophora and getting 200 and 200
Asking for Zoomastigophora and getting 200 and 200
Asking for Choanoflagellida and getting 200 and 200
Asking for Kinetoplastida and getting 200 and 200
Asking for Mollusca and getting 200 and 200
Asking for Polyplacophora and getting 200 and 200
Asking for Neoloricata and getting 200 and 200
Asking for Lepidopleurida and getting 200 and 200
Asking for Leptochitonidae and getting 200 and 200
Asking for Isc

/var/folders/7q/237gyf_x4yx4bblxcc0y91940000gp/T/ipykernel_69951/2811433694.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Common Name', 'Image URL']] = df[column_name].apply(lambda name: pd.Series(get_species_info_from_wikipedia(name)))
/var/folders/7q/237gyf_x4yx4bblxcc0y91940000gp/T/ipykernel_69951/2811433694.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Common Name', 'Image URL']] = df[column_name].apply(lambda name: pd.Series(get_species_info_from_wikipedia(name)))


,id,scientificNameID,acceptedNameUsageID,parentNameUsageID,namePublishedInID,scientificName,acceptedNameUsage,parentNameUsage,namePublishedIn,namePublishedInYear,...,modified,bibliographicCitation,references,license,rightsHolder,datasetName,institutionCode,datasetID,Common Name,Image URL
0,urn:lsid:marinespecies.org:taxname:1,urn:lsid:marinespecies.org:taxname:1,urn:lsid:marinespecies.org:taxname:1,NaN,NaN,Biota,Biota,Biota,NaN,NaN,...,2004-12-21,WoRMS (2024). Biota. Accessed at: https://www....,https://www.marinespecies.org/aphia.php?p=taxd...,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170,"Biota may refer to:\n\nBiota (biology), life, ...",Image not available
1,urn:lsid:marinespecies.org:taxname:2,urn:lsid:marinespecies.org:taxname:2,urn:lsid:marinespecies.org:taxname:2,urn:lsid:marinespecies.org:taxname:1,NaN,Animalia,Animalia,Biota,NaN,NaN,...,2004-12-21,WoRMS (2024). Animalia. Accessed at: https://w...,https://www.marinespecies.org/aphia.php?p=taxd...,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170,"Animals are multicellular, eukaryotic organism...",https://upload.wikimedia.org/wikipedia/commons...
2,urn:lsid:marinespecies.org:taxname:3,urn:lsid:marinespecies.org:taxname:3,urn:lsid:marinespecies.org:taxname:3,urn:lsid:marinespecies.org:taxname:1,NaN,Plantae,Plantae,Biota,NaN,NaN,...,2011-07-29,WoRMS (2024). Plantae. Accessed at: https://ww...,https://www.marinespecies.org/aphia.php?p=taxd...,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170,Plants are the eukaryotes that form the kingdo...,https://upload.wikimedia.org/wikipedia/commons...
3,urn:lsid:marinespecies.org:taxname:4,urn:lsid:marinespecies.org:taxname:4,urn:lsid:marinespecies.org:taxname:4,urn:lsid:marinespecies.org:taxname:1,NaN,Fungi,Fungi,Biota,NaN,NaN,...,2004-12-21,WoRMS (2024). Fungi. Accessed at: https://www....,https://www.marinespecies.org/aphia.php?p=taxd...,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170,A fungus (pl,https://upload.wikimedia.org/wikipedia/commons...
4,urn:lsid:marinespecies.org:taxname:5,urn:lsid:marinespecies.org:taxname:5,urn:lsid:marinespecies.org:taxname:5,urn:lsid:marinespecies.org:taxname:1,NaN,Protozoa,Protozoa,Biota,NaN,NaN,...,2011-07-29,WoRMS (2024). Protozoa. Accessed at: https://w...,https://www.marinespecies.org/aphia.php?p=taxd...,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170,Protozoa (sg,https://upload.wikimedia.org/wikipedia/commons...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,urn:lsid:marinespecies.org:taxname:194,urn:lsid:marinespecies.org:taxname:194,urn:lsid:marinespecies.org:taxname:194,urn:lsid:marinespecies.org:taxname:412646,NaN,Tergipedidae,Tergipedidae,Fionoidea,NaN,NaN,...,2017-09-28,MolluscaBase eds. (2024). MolluscaBase. Tergip...,https://www.marinespecies.org/aphia.php?p=taxd...,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170,"Tergipedidae is a family of sea slugs, aeolid ...",https://upload.wikimedia.org/wikipedia/commons...
96,urn:lsid:marinespecies.org:taxname:197,urn:lsid:marinespecies.org:taxname:197,urn:lsid:marinespecies.org:taxname:382243,urn:lsid:marinespecies.org:taxname:1057250,NaN,Basommatophora,Hygrophila,Tectipleura,NaN,NaN,...,2023-01-27,MolluscaBase eds. (2024). MolluscaBase. Basomm...,https://www.marinespecies.org/aphia.php?p=taxd...,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170,Basommatophora was a

In [26]:
import time

time_start = time.time()
limit_worms = worms.head(200)
# Apply the function to the DataFrame
updated_species_df = await asapply_wikipedia_info_to_df(limit_worms, "scientificName")

time_end = time.time()
time_elapsed = time_end - time_start
print(f"Time elapsed: {time_elapsed:.2f} seconds")
# Show the updated DataFrame
updated_species_df  # Run the asynchronous function in the Jupyter notebook

Time elapsed: 36.64 seconds


/var/folders/7q/237gyf_x4yx4bblxcc0y91940000gp/T/ipykernel_69951/1121254507.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Common Name', 'Image URL']] = pd.DataFrame(results, columns=['Common Name', 'Image URL'])
/var/folders/7q/237gyf_x4yx4bblxcc0y91940000gp/T/ipykernel_69951/1121254507.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Common Name', 'Image URL']] = pd.DataFrame(results, columns=['Common Name', 'Image URL'])


,id,scientificNameID,acceptedNameUsageID,parentNameUsageID,namePublishedInID,scientificName,acceptedNameUsage,parentNameUsage,namePublishedIn,namePublishedInYear,...,modified,bibliographicCitation,references,license,rightsHolder,datasetName,institutionCode,datasetID,Common Name,Image URL
0,urn:lsid:marinespecies.org:taxname:1,urn:lsid:marinespecies.org:taxname:1,urn:lsid:marinespecies.org:taxname:1,NaN,NaN,Biota,Biota,Biota,NaN,NaN,...,2004-12-21,WoRMS (2024). Biota. Accessed at: https://www.marinespecies.org/aphia.php?p=taxdetails&id=1,https://www.marinespecies.org/aphia.php?p=taxdetails&id=1,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170,"Biota may refer to:\n\nBiota (biology), life, living organisms\nBiota (ecology), biome, the plant and animal life of a region\nBiota (plant), or Platycladus orientalis, a coniferous tree\nBiota!, a proposed aquarium in London\nBiota, Cinco Villas, a municipality in Aragon, Spain\nBiota (band), an American experimental electronic music ensemble\nBiota (album), a 1982 album by the Mnemonist Orchestra, which later became Biota",Image not available
1,urn:lsid:marinespecies.org:taxname:2,urn:lsid:marinespecies.org:taxname:2,urn:lsid:marinespecies.org:taxname:2,urn:lsid:marinespecies.org:taxname:1,NaN,Animalia,Animalia,Biota,NaN,NaN,...,2004-12-21,WoRMS (2024). Animalia. Accessed at: https://www.marinespecies.org/aphia.php?p=taxdetails&id=2,https://www.marinespecies.org/aphia.php?p=taxdetails&id=2,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170,"Animals are multicellular, eukaryotic organisms in the biological kingdom Animalia ()",https://upload.wikimedia.org/wikipedia/commons/6/6f/Animal_diversity_b.png
2,urn:lsid:marinespecies.org:taxname:3,urn:lsid:marinespecies.org:taxname:3,urn:lsid:marinespecies.org:taxname:3,urn:lsid:marinespecies.org:taxname:1,NaN,Plantae,Plantae,Biota,NaN,NaN,...,2011-07-29,WoRMS (2024). Plantae. Accessed at: https://www.marinespecies.org/aphia.php?p=taxdetails&id=3,https://www.marinespecies.org/aphia.php?p=taxdetails&id=3,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170,Plants are the eukaryotes that form the kingdom Plantae; they are predominantly photosynthetic,https://upload.wikimedia.org/wikipedia/commons/thumb/5/57/Fr%C3%BChling_bl%C3%BChender_Kirschenbaum.jpg/1024px-Fr%C3%BChling_bl%C3%BChender_Kirschenbaum.jpg
3,urn:lsid:marinespecies.org:taxname:4,urn:lsid:marinespecies.org:taxname:4,urn:lsid:marinespecies.org:taxname:4,urn:lsid:marinespecies.org:taxname:1,NaN,Fungi,Fungi,Biota,NaN,NaN,...,2004-12-21,WoRMS (2024). Fungi. Accessed at: https://www.marinespecies.org/aphia.php?p=taxdetails&id=4,https://www.marinespecies.org/aphia.php?p=taxdetails&id=4,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170,A fungus (pl,https://upload.wikimedia.org/wikipedia/commons/thumb/f/fc/Fungi_collage.jpg/1024px-Fungi_collage.jpg
4,urn:lsid:marinespecies.org:taxname:5,urn:lsid:marinespecies.org:taxname:5,urn:lsid:marinespecies.org:taxname:5,urn:lsid:marinespecies.org:taxname:1,NaN,Protozoa,Protozoa,Biota,NaN,NaN,...,2011-07-29,WoRMS (2024). Protozoa. Accessed at: https://www.marinespecies.org/aphia.php?p=taxdetails&id=5,https://www.marinespecies.org/aphia.php?p=taxdetails&id=5,http://creativecommons.org/licenses/by/4.0/,WoRMS Editorial Board,World Register of Marine Species (WoRMS),VLIZ,https://doi.org/10.14284/170,Protozoa (sg,https://upload.wikimedia.org/wikipedia/commons/thumb/7/7c/Protozoa_collage_2.jpg/1024px-Protozoa_collage_2.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,urn:lsid:marinespecies.org:taxname:894,urn:lsid:marinespecies.org:taxname:894,urn:lsid:marinesp

In [27]:
pd.set_option("display.max_colwidth", None)
updated_species_df["Image URL"]

0                                                                                                                                               Image not available
1                                                                                        https://upload.wikimedia.org/wikipedia/commons/6/6f/Animal_diversity_b.png
2      https://upload.wikimedia.org/wikipedia/commons/thumb/5/57/Fr%C3%BChling_bl%C3%BChender_Kirschenbaum.jpg/1024px-Fr%C3%BChling_bl%C3%BChender_Kirschenbaum.jpg
3                                                              https://upload.wikimedia.org/wikipedia/commons/thumb/f/fc/Fungi_collage.jpg/1024px-Fungi_collage.jpg
4                                                    https://upload.wikimedia.org/wikipedia/commons/thumb/7/7c/Protozoa_collage_2.jpg/1024px-Protozoa_collage_2.jpg
                                                                                   ...                                                                             
195             